In [1]:
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.request import urlopen as uReq
import numpy as np
import pandas as pd
import re
from urllib import request
import timeit
from textblob import TextBlob
import itertools

In [6]:
#Enter number of pages to get data from
n_pages=50
soups=[]

#Location of my chromedriver
driver=webdriver.Chrome('C:\Softwares\chromedriver_win32\chromedriver.exe')

url1="https://www.business-standard.com"

#Fetching the URL and printing its title
driver.get(url1)
print(driver.title)

#Identifying the search area using the specific 'id' present in the html version of the page

search_text=driver.find_element_by_xpath('(//*[@id="head-suggest"])')
search_text.send_keys("Indian tourists in UK")

#Button accessed using the inspecting feature of Chrome and then copying the Xpath of the button to be clicked

search_button=driver.find_element_by_xpath('(//*[@id="search_btn"]/input)')
search_button.click()


current_url=driver.current_url
soups.append(BeautifulSoup(driver.page_source,'html.parser'))

#Click on the 'next' button to go to next page, fetch its data and then store it in a soup list
while(n_pages!=0):
    if(n_pages==50):
        next_button=driver.find_element_by_xpath('(//*[@id="hpcontentbox"]/div[3]/div[2]/div[1]/div[2]/div/div[1]/a)')
    else:
        next_button=driver.find_element_by_xpath('(//*[@id="hpcontentbox"]/div[3]/div[2]/div[1]/div[2]/div/div[2]/a)')
        
    next_button.click()
    soups.append(BeautifulSoup(driver.page_source,'html.parser'))
    n_pages-=1
#Pause for 10 seconds
time.sleep(5)

#Quits the browsed page
driver.quit()

India News, Latest News Headlines, BSE live, NSE Live, Stock Markets Live, Financial News, Business News & Market Analysis on Indian Economy - Business Standard


### Functions used are defined in the next cell

In [33]:
#This function separates the information on the basis of date, heading, topic, article description and link for the article 

def infoCollect(containers):
    links=[]
    heading=[]
    description=[]
    date=[]
    topic=[]
    for c in containers[0].find_all('li',{'class':''}):
        links.append(url1+c.find_all('a')[1]['href'])
        heading.append(c.find_all('a')[1].get_text())
        date.append(c.p.get_text().split('|')[0])
        topic.append(c.p.get_text().split('|')[1])
        if(len(c.find_all('p'))==2):
            description.append(c.find_all('p')[1].get_text())
        elif(len(c.find_all('p'))==4):
            description.append(c.find_all('p')[2].get_text())
    return date,heading,topic,links,description

#Content Fetch using the Links saved in infoCollect function

def findText(containers,links):
    #elements=int(len(containers[0].find_all('p'))/2)
    elements=19
    text=[]
    fintext=[]
    for i in range(0,elements):
        tempclient=uReq(l[i]) #Opens connection to website and downloads the page
        page_temp=tempclient.read() #Dont run this directly
        tempclient.close() #CLoses the page to prevent crashing of console
        temp_soup=BeautifulSoup(page_temp, "html.parser")
        cont=temp_soup.find_all("span",{'class':'p-content'})
        for x in cont[0].find_all("p",{"class":''}):
            [x.extract() for x in x.findAll('script')]
            text.append(x.get_text())
        x=' '.join(text)
        del text[:]
        fintext.append(x)
    return fintext

#Fetches the containers(only the part that we need) based on initial soup html for each page
def containerFetch(soup):
    #containers=soup.find_all('div',{'class':"listing-main topB mT10"})
    containers=[]
    for i in range(0,len(soup)-1):
        containers.append((soup[i]).find_all('div',{'class':'listing-main topB mT10'}))
    return containers

#Formats the output from infoCollect and finText

def listFormat(date,topic,links,description,fintext):
    #nhead=list(itertools.chain.from_iterable(heading))
    ndate=list(itertools.chain.from_iterable(date))
    ntopic=list(itertools.chain.from_iterable(topic))
    ndesc=list(itertools.chain.from_iterable(description))
    nfin=list(itertools.chain.from_iterable(fintext))
    for i in range(0,len(links)-1):
        if(len(links[i])>19):
            del links[i][-1]
    nlinks=list(itertools.chain.from_iterable(links))
    return ndate,ntopic,nlinks,ndesc,nfin

In [9]:
#Main Code
start_time = timeit.default_timer()
all_containers=containerFetch(soups)
#heading=[]
date=[]
topic=[]
links=[]
description=[]
fintext=[]
p_count=0
for i in range(0,len(all_containers)-1):
    start_time = timeit.default_timer()
    da,h,t,l,de=infoCollect(all_containers[i])
    ft=findText(all_containers[i],l)
    p_count+=1
    print("Page number :",p_count)
    #heading.append(h)
    date.append(da)
    topic.append(t)
    links.append(l)
    description.append(de)
    fintext.append(ft)
    elapsed = timeit.default_timer() - start_time
    print("Time taken for fetching page : ",elapsed)
    print('---------------------------------------')
elapsed = timeit.default_timer() - start_time
print(elapsed)

Page number : 1
Time taken for fetching page :  19.879553377600388
---------------------------------------
Page number : 2
Time taken for fetching page :  18.65702781178462
---------------------------------------
Page number : 3
Time taken for fetching page :  16.507239096697262
---------------------------------------
Page number : 4
Time taken for fetching page :  19.37739160412694
---------------------------------------
Page number : 5
Time taken for fetching page :  21.655058322372767
---------------------------------------
Page number : 6
Time taken for fetching page :  28.462000573438857
---------------------------------------
Page number : 7
Time taken for fetching page :  24.30022254889022
---------------------------------------
Page number : 8
Time taken for fetching page :  23.55398972361158
---------------------------------------
Page number : 9
Time taken for fetching page :  21.75945150118099
---------------------------------------
Page number : 10
Time taken for fetching p

In [34]:
ndate,ntopic,nlinks,ndesc,nfin=listFormat(date,topic,links,description,fintext)

In [36]:
#Combines the data based on date,topic,description,link for the article,content
combine={'Date':ndate,'Topic':ntopic,'Description':ndesc,'Link for website':nlinks,'Content':nfin}
data=pd.DataFrame(combine)
col=['Description','Content']
for c in col:
    data[c]=data[c].str.replace('\r',' ')
data.Content = data.Content.apply(lambda x:re.sub(' +',' ',x))
data.Content = data.Content.apply(lambda x:x.strip("/"))
data.Content = data.Content.apply(lambda x:x.strip())
data['Sentiment']=data['Content'].apply(lambda x:TextBlob(x).sentiment[0])

In [37]:
data.head()

,Date,Topic,Description,Link for website,Content,Sentiment
0,"March 10, 2018, Saturday",Economy & Policy,Said new UK-India Visa Agreement being propose...,https://www.business-standard.com/article/econ...,A leading UK-based think tank has released a r...,0.214417
1,"October 09, 2017, Monday",Current Affairs,China on Sunday referred to the 1890 UK-China ...,https://www.business-standard.com/article/curr...,Doklam and the Sikkim stretch of the border be...,0.045455
2,"August 12, 2011, Friday",Economy & Policy,"Indian travellers, both for leisure and busine...",https://www.business-standard.com/article/econ...,"Indian travellers, both for leisure and busine...",0.134702
3,"March 25, 2011, Friday",Economy & Policy,"Britain's tourism promotion board, Visit Brita...",https://www.business-standard.com/article/econ...,"Britain's tourism promotion board, Visit Brita...",0.110638
4,"August 12, 2010, Thursday",Economy & Policy,Prime Minister David Cameron today said visa ...,https://www.business-standard.com/article/econ...,Prime Minister David Cameron today said visa r...,0.000000


In [38]:
data['Content'][0]

'A leading UK-based think tank has released a research to support its call for a new, more economical visa regime to attract Indian visitors to the country. \nThe Royal Commonwealth Society (RCS) revealed that the UK was losing out as neighbouring France attracted 185,000 more Indian business visitors and tourists in 2016.\n\n\n\n \nOverall the number of visits to the UK by Indian nationals fell by 1.73 per cent in 2016, while in France it grew by 5.3 per cent. \nThe UK\'s market share of Indian outbound tourists has more than halved from 4.4 per cent in 2006 to 1.9 per cent in 2016 600,000 Indians visited France in 2016, 185,000 more than visited the UK, RCS says in its new Britain and India: Building a New Visa Partnership\' fact sheet presented to British MPs earlier this week as part of its campaign launched in 2016 for a new UK-India bilateral visa agreement which would considerably reduce the cost of tourist visas. \nThe new UK-India Visa Agreement being proposed would reduce the

In [40]:
data.shape

(931, 6)

In [41]:
data.to_csv('Business_Insider_Dataset.csv')